### Code Chunker

- Split code into chunks based on code structure

- It splits code into chunks based on its structure, leveraging Abstract Syntax Trees (ASTs) to create contextually relevant segments.

AST is a tree representation of the syntactic structure of source code, \
providing a hierarchial view of code elements like functions, variables, and control flows.

In [4]:
from chonkie import CodeChunker

# Basic initialization with default parameters
chunker = CodeChunker(
    language="python",                 # Specify the programming language
    tokenizer_or_token_counter="gpt2", # Tokenizer to use
    chunk_size=250,                    # Maximum tokens per chunk
    include_nodes=False                # Optionally include AST nodes in output
)

In [5]:
code = """
mlflow.set_tracking_uri('http://localhost:8080')
mlflow.set_experiment('Exp-1')

with mlflow.start_run() as run:
    x, y = make_regression(n_features=4, n_informative=2, random_state=42, shuffle=False)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    params = {'max_depth': 1, 'random_state': 42}
    model = RandomForestRegressor(**params)
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    signature = infer_signature(x_test, y_pred)
    
    mlflow.log_params(params)
    mlflow.log_metrics({'mse': mean_squared_error(y_test, y_pred)})

    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path='sklearn-model-2',
        signature=signature,
        registered_model_name='sklearn-random-forest-reg'
    )

"""
chunks = chunker.chunk(code)

for chunk in chunks:
    print(f"Chunk text: {chunk.text}")
    print(f"Token count: {chunk.token_count}")
    print(f"Language: {chunk.language}")
    if chunk.nodes:
        print(f"Node count: {len(chunk.nodes)}")

Chunk text: 
mlflow.set_tracking_uri('http://localhost:8080')
mlflow.set_experiment('Exp-1')

with mlflow.start_run() as run:
    x, y = make_regression(n_features=4, n_informative=2, random_state=42, shuffle=False)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    params = {'max_depth': 1, 'random_state': 42}
    model = RandomForestRegressor(**params)
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    signature = infer_signature(x_test, y_pred)
    
    mlflow.log_params(params)
    mlflow.log_metrics({'mse': mean_squared_error(y_test, y_pred)})

    
Token count: 222
Language: None
Chunk text: mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path='sklearn-model-2',
        signature=signature,
        registered_model_name='sklearn-random-forest-reg'
    )


Token count: 85
Language: None


### Semantic Chunking

Split text into chunks based on semantic similarity

The SemanticChunker splits text into chunks based on semantic similarity, ensuring that related content stays together in the same chunk. \
This approach is particularly useful for RAG applications where context preservation is crucial.

In [6]:
from chonkie import SemanticChunker

# Basic initialization with default parameters
chunker = SemanticChunker(
    embedding_model="minishlab/potion-base-8M",  # Default model
    threshold=0.5,                               # Similarity threshold (0-1) or (1-100) or "auto"
    chunk_size=20,                              # Maximum tokens per chunk
    min_sentences=1                              # Initial sentences per chunk
)

In [7]:
text = """First paragraph about a Donkeys.
Why donkey are colored black and white.
This paragraph is about Pigs.
Pigs are pink and have curly tails."""

chunks = chunker.chunk(text)

for chunk in chunks:
    print(f"Chunk text: {chunk.text}")
    print(f"Token count: {chunk.token_count}")
    print(f"Number of sentences: {len(chunk.sentences)}")


Chunk text: First paragraph about a Donkeys.
Why donkey are colored black and white.

Token count: 15
Number of sentences: 2
Chunk text: This paragraph is about Pigs.
Pigs are pink and have curly tails.
Token count: 14
Number of sentences: 2


### SDPM Chunker

Split text using Semantic Double-Pass Merging for improved context preservation

The SDPMChunker extends semantic chunking by using a double-pass merging approach. \
It first groups content by semantic similarity, then merges similar groups within a skip window, \
allowing it to connect related content that may not be consecutive in the text. This technique is \
particularly useful for documents with recurring themes or concepts spread apart.

In [ ]:
from chonkie import SDPMChunker

# Basic initialization with default parameters
chunker = SDPMChunker(
    embedding_model="minishlab/potion-base-8M",  # Default model
    threshold=0.5,                              # Similarity threshold (0-1)
    chunk_size=50,                             # Maximum tokens per chunk
    min_sentences=1,                            # Initial sentences per chunk
    skip_window=1                               # Number of chunks to skip when looking for similarities
)

In [8]:
text = """The neural network processes input data through layers.
Training data is essential for model performance.
GPUs accelerate neural network computations significantly.
Quality training data improves model accuracy.
TPUs provide specialized hardware for deep learning.
Data preprocessing is a crucial step in training."""

chunks = chunker.chunk(text)

for chunk in chunks:
    print(f"Chunk text: {chunk.text}")
    print(f"Token count: {chunk.token_count}")
    print(f"Number of sentences: {len(chunk.sentences)}")
    print('\n')

Chunk text: The neural network processes input data through layers.
Training data is essential for model performance.

Token count: 17
Number of sentences: 2


Chunk text: GPUs accelerate neural network computations significantly.
Quality training data improves model accuracy.

Token count: 16
Number of sentences: 2


Chunk text: TPUs provide specialized hardware for deep learning.

Token count: 9
Number of sentences: 1


Chunk text: Data preprocessing is a crucial step in training.
Token count: 12
Number of sentences: 1




### Late Chunker

Split text into chunks based on a late-bound token count

The LateChunker implements the late chunking strategy described in the Late Chunking paper. \
It builds on top of the RecursiveChunker and uses document-level embeddings to create more \
semantically rich chunk representations.

Instead of generating embeddings for each chunk independently, the LateChunker first encodes \
the entire text into a single embedding. It then splits the text using recursive rules and derives \
each chunk’s embedding by averaging relevant parts of the full document embedding. This allows \
each chunk to carry broader contextual information, improving retrieval performance in RAG systems.

In [10]:
# !pip install numpy==1.26.4
# !pip install tf-keras

In [10]:
from chonkie import LateChunker, RecursiveRules

chunker = LateChunker(
    embedding_model="all-MiniLM-L6-v2",
    chunk_size=10,
    rules=RecursiveRules(),
    min_characters_per_chunk=24,
)

2025-06-22 19:12:13.236055: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750599733.739143   11165 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750599733.862928   11165 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750599734.913775   11165 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750599734.913825   11165 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750599734.913829   11165 computation_placer.cc:177] computation placer alr

In [11]:
text = """First paragraph about a specific topic.
Second paragraph continuing the same topic.
Third paragraph switching to a different topic.
Fourth paragraph expanding on the new topic."""

chunks = chunker(text)

for chunk in chunks:
    
    print(f"Chunk text: {chunk.text}")
    print(f"Token count: {chunk.token_count}")
    print('\n')
    #print(f"Number of sentences: {len(chunk.sentences)}")

Chunk text: First paragraph about a specific topic.

Token count: 8


Chunk text: Second paragraph continuing the same topic.

Token count: 7


Chunk text: Third paragraph switching to a different topic.

Token count: 8


Chunk text: Fourth paragraph expanding on the new topic.
Token count: 9




### Neural Chunker

Split text using a fine-tuned BERT model to detect semantic shifts

The NeuralChunker leverages the power of deep learning! It uses a fine-tuned BERT \
model specifically trained to identify semantic shifts within text, allowing it to \
split documents at points where the topic or context changes significantly. This \
provides highly coherent chunks ideal for RAG.

In [12]:
from chonkie import NeuralChunker

# Basic initialization with default parameters
chunker = NeuralChunker(
    model="mirth/chonky_modernbert_base_1",  # Default model
    device_map="cuda",                        # Device to run the model on ('cpu', 'cuda', etc.)
    min_characters_per_chunk=10,             # Minimum characters for a chunk
    return_type="chunks"                     # Output type
)

Device set to use cuda


In [13]:
text = """Limited context window - All LLMs have a limit on how much text they can process at once. This is referred to as the Context Window. Chunking helps in breaking down the large text document into processable tokens
Computational Efficiency - It is not possible to load a 100GB document every time you make a query. Attention mechanisms, even when optimized, are computationally expensive O(n). Chunking keeps things efficient and memory-friendly.
Better Representation - As mentioned earlier, chunks represent each idea as an independent entity. Not chunking your document will likely cause your model to conflate concepts and get confused. 
Representation models use lossy compression, so keeping chunks concise ensures the model understands the context better.
Reduced Hallucination - Feeding too much context at once causes the models to hallucinate. They start using irrelevant information to answer queries, and that's a big no-no. Smaller, focused chunks reduce this risk."""

chunks = chunker.chunk(text)

for chunk in chunks:
    print(f"Chunk text: {chunk.text}")
    print(f"Token count: {chunk.token_count}") # Note: token_count might be added post-hoc or not available depending on implementation
    print(f"Start index: {chunk.start_index}")
    print(f"End index: {chunk.end_index}")
    print('\n')

W0622 19:14:46.607000 11165 torch/_inductor/utils.py:1137] [1/0] Not enough SMs to use max_autotune_gemm mode


Chunk text: Limited context window - All LLMs have a limit on how much text they can process at once. This is referred to as the Context Window. Chunking helps in breaking down the large text document into processable tokens

Token count: 48
Start index: 0
End index: 213


Chunk text: Computational Efficiency - It is not possible to load a 100GB document every time you make a query. Attention mechanisms, even when optimized, are computationally expensive O(n). Chunking keeps things efficient and memory-friendly.

Token count: 51
Start index: 213
End index: 445


Chunk text: Better Representation - As mentioned earlier, chunks represent each idea as an independent entity. Not chunking your document will likely cause your model to conflate concepts and get confused. 
Representation models use lossy compression, so keeping chunks concise ensures the model understands the context better.

Token count: 60
Start index: 445
End index: 761


Chunk text: Reduced Hallucination - Feeding too much